In [15]:
#Dependencies
from openpyxl import load_workbook
import openpyxl as pyxl
from pathlib import Path
import os
# assign directory
directory = 'files'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'files'

In [16]:
#This cell can be used for debugging to confirm whether or not the path you've entered is valid. 
Path(r"C:\Invoices").exists()

True

In [17]:
master=[] #Will contain the data we're aggregating

In [18]:
#Option 1: Going to only specified files

In [31]:
#Helper function to build path strings when iterating through multiple files
def pthBld(flNm,flExtn="",path_building_variable=('Paid',0)):
    """Take a file name and expected file type, and based on some other input, add the rest of the file path for reference by workbook opening function"""
    if flExtn!="": flExtn="."+flExtn
    tp=path_building_variable[0]
    yr=path_building_variable[1]
    if yr==22:
        return (r"C:\Invoices\Invoices 2022"+chr(92)+tp+chr(92)+flNm+flExtn)
    else:
        return (r"C:\Invoices\Invoices '"+chr(92)+tp+chr(92)+str(yr)+chr(92)+flExtn)

In [34]:
#Test function. Output will have double slashes due to vagaries of paths/strings in this language etc but if exists() yields True, the path is good
p=pthBld('wow',"xlsx",('Paid',22))
p, Path(p).exists()

('C:\\Invoices\\Invoices 2022\\Paid\\wow.xlsx', False)

In [ ]:
#Option 2: Searching for every file in a given directory based on name structure

In [12]:
for pr in pairs:
    wb = load_workbook(filename = pr[0],data_only=True)
    print(pr[0])
    for i in range(6,75): #All rows
        if 'search_txt' in str(wb['sheet1'].cell(row=i,column=9).value).lower():
            master.append(str(wb['sheet1'].cell(row=i,column=3).value)+'_'+str(pr[1]))

In [13]:
len(master),master

(0, [])

In [14]:
wb = pyxl.Workbook()
ws = wb.active
for id in master:
    ws.cell(row=2+master.index(id),column=1).value=id
wb.save(r"C:\Invoices\Aggregated_Data.xlsx")
wb.close()